In [2]:
"""
Queries each sudo pool for pair specific attributes
to build local csv of all pair pools.
"""
# since pools are not verified on Etherscan, need to use local contracts
# switch to sudo-ape wd first for ape project to work properly
import os
from pathlib import Path

if Path(*Path(os.getcwd()).parts[-3:]) == Path('sudo-ape/notebook/queries'):
    os.chdir('../..')

In [5]:
import pandas as pd
import typing as tp

from ape import accounts, chain, Contract, networks, project
from math import ceil

In [6]:
networks.parse_network_choice('ethereum:mainnet:alchemy').__enter__() 

<alchemy chain_id=1>

In [9]:
# check LSSVMPair in local project
project.LSSVMPair

<LSSVMPair>

In [10]:
# load sudo pairs from csv output of ./sudo-pairs.ipynb
query_pairs = pd.read_csv('notebook/data/pairs.csv')

In [11]:
query_pairs

,event_name,contract_address,event_arguments,transaction_hash,block_number,block_hash,log_index,transaction_index,poolAddress
0,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x08142348E6BBf233002b81047bc...,0xba75dd0b017667074a1ed080571fee06b3cea5444f25...,14650748,0xf63250e045af2d83fca5535fe3738deffd3c6f17f93b...,15,9,0x08142348E6BBf233002b81047bc2f27026af10A5
1,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x518b7160990AD19dcfF0Ba028e2...,0x7e33fe2f08ee09a339b127c8658a892600637da3448d...,14652189,0x4b0de468eecdfea07b64168655f129d43d8e56f01f42...,41,37,0x518b7160990AD19dcfF0Ba028e245fA616b44F5E
2,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x351E40e39ae7D0d60Dc283485bC...,0x230624db7d760f3ad189bdebb431ef74b6cd899855d6...,14652199,0x572e74e542e87e141a2176725d7b216133bd0d13dcf5...,150,117,0x351E40e39ae7D0d60Dc283485bC627891Ec1D00E
3,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x0636a6Bd07dDBEA28a3cc63353e...,0xff03f1a4956f878f1bd42a6dfa2ec18165822cde3d15...,14652223,0x877d4993fe635aaaea71fa3c8bb8c705d67f86559c3e...,142,108,0x0636a6Bd07dDBEA28a3cc63353ea725e8cE509e2
4,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x866C5c02dC07EE37146c4874d70...,0x0c2b148a424bb8f25f32ca3ded2c87704cb73ab5b65b...,14662427,0x8f687e0ea159efeb37fd7e65ec00a008e6b96cc1b7ee...,171,149,0x866C5c02dC07EE37146c4874d704D3603bF5D950
...,...,...,...,...,...,...,...,...,...
30990,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x9AAa20179145C9153a36Bbf71e2...,0x35feb962e106bfbd16bb949605d171bda649c015621d...,15561562,0x7fe005a4046800acc48f09ae4a0c92eea79f0cf0a38f...,481,265,0x9AAa20179145C9153a36Bbf71e2FF0c6eC2bed1f
30991,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0xf29EE6A9C566Fb837988a72bdaa...,0xf8451b200c08f1603db95f39140fb21d633f6119e9f4...,15561651,0x3c1e8ab50f8a8dcba3e5fae236eb5ec6640bf942d0e3...,146,92,0xf29EE6A9C566Fb837988a72bdaa5749564Dc8b69
30992,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0xCbb272D3c518386D1dd5a673698...,0x5eb463db486d91a5ee20d64f894d504b03820bcde553...,15561725,0x03a607aa73430521c37d30a177c1de30509b46a143fd...,110,39,0xCbb272D3c518386D1dd5a673698F02d1a0063D3c
30993,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0xC66Cfab72F2d43eb6Bb7058B2af...,0xe482845e9b8234d98b0e6ed1747b62bafc6ebc345dad...,15561734,0x6889adf6bf42faf501840a41858d7687860edc3e1138...,81,72,0xC66Cfab72F2d43eb6Bb7058B2af484A45eB2b9E9


In [12]:
# filter out non-TRADE type pools to make querying easier, since we only care
# about collections with some trade liquidity.
# poolType == 2 is TRADE
sudo_pair_xmons = Contract("0x5caf332dca4e6c9e69d52f320c21e74845353db0")
sudo_pair_xmons.poolType()

2

In [13]:
# NewPair events only give the pair address without any addition. Query
# for additional pool info from each pool contract at poolAddress in NewPair
def get_pair_specifics(key: str, x: pd.Series, is_method: bool = True) -> tp.Any:
    """
    Gets pair specific info from the LSSVMPair contract.
    """
    addr = x['poolAddress']
    pair = project.LSSVMPair.at(addr)
    val = getattr(pair, key)() if is_method else getattr(pair, key)
    return val

In [14]:
# start out with poolType info to join into query_pairs df
# chunk requests since otherwise issues with alchemy
chunk_size = 300
num_chunks = ceil(len(query_pairs) / chunk_size)

In [15]:
# read in poolType chunks we already fetched and stored in csv (so don't have to query as many to catchup)
query_pool_types = pd.read_csv('notebook/data/specifics/pool_types.csv')

In [20]:
query_pool_types

,poolType
0,1
1,1
2,1
3,1
4,1
...,...
21295,1
21296,1
21297,1
21298,1


In [53]:
# determine the chunk id currently on given csv file load ...
idx_chunk = int(len(query_pool_types) / chunk_size)

In [71]:
# repeat execution of this until get specifics on poolType to filter out poolType!=2 pools
if idx_chunk < num_chunks+1:
    %time query_pairs_pool_chunk = query_pairs[chunk_size*idx_chunk:chunk_size*(idx_chunk+1)].apply(lambda x: get_pair_specifics('poolType', x), axis=1)
    query_pool_types = pd.concat([query_pool_types, pd.DataFrame(data={'poolType': query_pairs_pool_chunk})])
    idx_chunk += 1
    del query_pairs_pool_chunk  # del the chunk so no issues with manual loop

CPU times: user 1min 48s, sys: 18.5 s, total: 2min 7s
Wall time: 10min 40s


In [72]:
idx_chunk

74

In [73]:
query_pool_types

,poolType
0,1
1,1
2,1
3,1
4,1
...,...
22195,1
22196,1
22197,1
22198,1


In [74]:
# reduces number of pools by factor of ~ 10
query_pool_types[query_pool_types.poolType == 2]

,poolType
6,2
8,2
10,2
11,2
12,2
...,...
22155,2
22164,2
22173,2
22184,2


In [75]:
# save temp to csv for easy loading above in case mess up in nb
query_pool_types.to_csv('notebook/data/specifics/pool_types.csv', index=False)